In [ ]:
import numpy as np
import pandas as pd
import os
import tifffile
import glob
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec15
import matplotlib.colors as colors
import matplotlib.lines as mlines
from matplotlib_venn import venn3

import scipy.cluster.hierarchy as sch
from scipy.stats import pearsonr
from scipy.cluster.hierarchy import dendrogram, linkage
import seaborn as sns
import random
from sklearn.cluster import AgglomerativeClustering
import cmasher


import sklearn
from sklearn import linear_model
from sklearn.metrics import r2_score 
from sklearn.cluster import KMeans

import warnings
from scipy.stats import percentileofscore
warnings.filterwarnings('ignore')

In [ ]:
main_dir_list = ["D:\\Neural_25x_motor_20231014\\20231014"]*3+["D:\\Neural_25x_motor_20231014\\20231015"]*4
fish_index_list = ['F4_17_00','F5_19_00','F6_21_00','F3_15_00','F4_17_00','F5_19_00','F6_20_30']

visuomotor_list = ['ns','dot4','dot15','looming','motor_Freezing','motor_PC','motor_escape']


sensory_list = ['dot15','looming','dot4']
sensory_color_list = ['#F88379','#ccff00','#09D0EF']
sensory_motor_list = ['dot15_motor_Freezing','looming_motor_escape','dot4_motor_PC']
motor_color_list = ['crimson','lime','dodgerblue']


time_len = 44
plane_no = 14
trial_no=48
fps = 2
fps_behavior = 200

output_dir = 'F:\\freezing_project\\'

df_VMV_7fish = pd.read_csv(output_dir+'\\df_VMV_8fish_v6.csv')
np_trace_no_esc = np.load(output_dir+'\\trace_all_v6.npy')

df_corr = pd.DataFrame(columns = df_VMV_7fish.columns,index = df_VMV_7fish.index)
df_corr.ROI_id = df_VMV_7fish.ROI_id.copy()
df_corr.Date = df_VMV_7fish.Date.copy()
df_corr.Fish = df_VMV_7fish.Fish.copy()
df_corr.Plane = df_VMV_7fish.Plane.copy()

df_corr.index = range(0,df_corr.shape[0])


reg_x = []
reg_y = []
reg_z = []
for i in df_corr.index:
#     print(i)
    date_no = df_corr['Date'][i]
    fish = df_corr['Fish'][i]
    plane = df_corr['Plane'][i]
    ROI_id = df_corr['ROI_id'][i]
    df_cell_info = pd.read_csv('D:\\Neural_25x_motor_20231014\\'+str(date_no)+'\\processed\\'+fish+'\\processed_suite2p\\plane'+str(plane)+'\\DeltaF_F\\cell_info_'+fish+'_plane'+str(plane)+'_reg.csv')

    cell_idx = np.where(df_cell_info.ROI_id == ROI_id)[0][0]
    reg_x.append(df_cell_info.reg_x[cell_idx])
    reg_y.append(df_cell_info.reg_y[cell_idx])
    reg_z.append(df_cell_info.reg_z[cell_idx])
df_corr['reg_x'] = reg_x
df_corr['reg_y'] = reg_y
df_corr['reg_z'] = reg_z


In [ ]:
def stars(p):
    if p < 0.0001:
        return "****"
    elif (p < 0.001):
        return "***"
    elif (p < 0.01):
        return "**"
    elif (p < 0.05):
        return "*"
    else:
        return "n.s."

def distance(x, y, x0, y0):
    d_x = x - x0
    d_y = y - y0
    dis = np.sqrt( d_x**2 + d_y**2 )
    return dis

def getOverlap(a, b):
     return max(0, min(a[1], b[1]) - max(a[0], b[0]))
def reject_outliers(data, m = 3.5):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else np.zero(len(d))
    return data[s<m]

def gen_regressor(x, tau, fps=2):
    duration = int(length/fps)
    
    t = np.arange(0, duration, duration/len(x))
    if tau == 0:
        return x
    zeros = np.zeros_like(t)
    padded = np.concatenate([zeros,x,zeros])

    kernel = np.concatenate([zeros, np.exp(-t/ tau )])

    convolved = np.convolve(padded, kernel, mode='same')[len(t):-len(t)]    
    if convolved.mean() == 0:
        return convolved
    return convolved / convolved.mean()

def get_regression(df0f,regressor):
    listregintercept=[]
    listregcoeff=[]
    listr2=[] 

    if regressor.ndim == 1:
        regressors = {'Dot': regressor}
        X = pd.DataFrame(regressors,columns=['Dot']) 


        for i in range(0,len(df0f)):
            df = pd.DataFrame (df0f[i])     
            regr = linear_model.LinearRegression()           
            x = regr.fit(X,df)
            listregcoeff.append(regr.coef_[0][0])  
            listregintercept.append(x.intercept_[0])
            r2 = r2_score(df, x.predict(X), multioutput='raw_values')
            listr2.append(r2[0])
    if regressor.ndim == 2:
        if len(df0f)!=regressor.shape[0]:
            print('error in array size')
            return -1
        for i in range(0,len(df0f)):
            regressors = {'Dot': regressor[i]}
            X = pd.DataFrame(regressors,columns=['Dot']) 
            df = pd.DataFrame (df0f[i])     
            regr = linear_model.LinearRegression()           
            x = regr.fit(X,df)
            listregcoeff.append(regr.coef_[0][0])  
            listregintercept.append(x.intercept_[0])
            r2 = r2_score(df, x.predict(X), multioutput='raw_values')
            listr2.append(r2[0])
            
    return listregintercept,listregcoeff,listr2


liststim = np.concatenate([np.zeros(18),np.ones(8),np.zeros(18)])
length =len(liststim)
print(liststim.shape)

sensory_regressor= gen_regressor(liststim,1.8)
sensory_regressor = (sensory_regressor-min(sensory_regressor))/(max(sensory_regressor)-min(sensory_regressor))

plt.plot(sensory_regressor,color='navy',linewidth = 3)
plt.fill_between(np.arange(18,26),1.5,-0.2,linewidth = 1,color = 'lightcoral',alpha = 0.2)
# plt.axis('off')
plt.show()

In [ ]:
# regressor building test
time_len = 44
PC_start = np.array([900])
Freezing_interval = np.array([(300,500),[1900,2200]])


PC_start = PC_start/100*2
PC_start = PC_start.astype(int)

listPC = np.zeros(time_len)
for motor_i in PC_start:
    listPC[motor_i:motor_i+1] = 1



Freezing_interval = Freezing_interval/200*2
Freezing_interval = Freezing_interval.astype(int)

listFreezing = np.zeros(time_len)
for motor_i_start,motor_i_end in zip(Freezing_interval[:,0],Freezing_interval[:,1]):
    listFreezing[motor_i_start-1:motor_i_end-1] = 1
    
motor_regressor_test = gen_regressor(listPC,7)
motor_regressor_test = (motor_regressor_test-min(motor_regressor_test))/(max(motor_regressor_test)-min(motor_regressor_test))

plt.plot(motor_regressor_test,color='navy',linewidth = 3)
plt.axis('off')
plt.ylim(-0.1,1.1)
plt.axvline(18,linestyle = '--',color = 'coral',linewidth = 3)
plt.show()

# sensory average and trace surplus calculation

In [ ]:
np_motor_surplus_trace = np_trace_no_esc.copy()
np_motor_avg_trace = np_trace_no_esc.copy()

for main_dir,fish in zip(main_dir_list,fish_index_list):
    date_no = main_dir.split('\\')[-1]
    idx_fish = np.where(np.logical_and(df_corr['Date'] == int(date_no),df_corr['Fish'] == fish))[0]
     # read bradyinfo and vsinfo
    vsinfo = pd.read_excel(main_dir+'\\behavior\\'+fish+'//vsinfo_'+date_no+'_'+fish+'.xlsx')
    bradyinfo_dir = glob.glob(main_dir+'\\behavior\\'+fish+'\\Bradyinfo*'+fish+'.xlsx')[0]
                                  
    bradyinfo = pd.read_excel(bradyinfo_dir,sheet_name = 'Bradyinfo')
    trial_no = vsinfo.shape[0]
    
    roi = eval(bradyinfo['roi'][0])
    total_pixel_no = roi[2]*roi[3]
    
    
      # every VMV's trial index of this fish
    for ii in range(0,4):
        locals()['Index_'+str(visuomotor_list[ii])] = []
    for t in range(0,trial_no):
        if bradyinfo['Good_Pixel_Number'][t]/total_pixel_no>0.05:
            if vsinfo.Left_Stimulus_Type[t] == 'n' and vsinfo.Right_Stimulus_Type[t] == 'n':
                Index_ns.append(t+1)
            else:
                if vsinfo.Left_Stimulus_Type[t] == 'm' or vsinfo.Right_Stimulus_Type[t] == 'm':
                    if vsinfo.Left_Stimulus_Size[t] + vsinfo.Right_Stimulus_Size[t] == 15:
                        Index_dot15.append(t+1)
                    if vsinfo['Left_Color'][t]=='UV' or vsinfo['Right_Color'][t]=='UV':
                        Index_dot4.append(t+1)
                if vsinfo['Left_Stimulus_Type'][t]=='l' or vsinfo['Right_Stimulus_Type'][t]=='l':
                    Index_looming.append(t+1)
    
    for VMV in visuomotor_list[1:4]:
        np_motor_avg_VMV_temp = np.zeros((idx_fish.shape[0],time_len))
        for t in locals()['Index_'+VMV]:
            np_motor_avg_VMV_temp = np.add(np_motor_avg_VMV_temp,np_trace_no_esc[idx_fish,t*time_len-time_len:t*time_len]/len(locals()['Index_'+VMV]))
        for t in locals()['Index_'+VMV]:
            np_motor_avg_trace[idx_fish,t*time_len-time_len:t*time_len] = np_motor_avg_VMV_temp
            np_motor_surplus_trace[idx_fish,t*time_len-time_len:t*time_len] = np.subtract(np_trace_no_esc[idx_fish,t*time_len-time_len:t*time_len],np_motor_avg_VMV_temp)
        

# SI calculation and sensory neuron selection

In [ ]:
# calculate the sensory idx of each ROI to draw scatter plot
df_corr['sensory_idx_dot15'] = 0
df_corr['sensory_idx_dot4'] = 0
df_corr['sensory_idx_looming'] = 0


for main_dir,fish in zip(main_dir_list,fish_index_list):
    date_no = main_dir.split('\\')[-1]
    print(date_no+fish)
    idx_fish = np.where(np.logical_and(df_corr['Date'] == int(date_no),df_corr['Fish'] == fish))[0]
    
    # read bradyinfo and vsinfo
    vsinfo = pd.read_excel(main_dir+'\\behavior\\'+fish+'//vsinfo_'+date_no+'_'+fish+'.xlsx')
    bradyinfo_dir = glob.glob(main_dir+'\\behavior\\'+fish+'\\Bradyinfo*'+fish+'.xlsx')[0]
                                  
    bradyinfo = pd.read_excel(bradyinfo_dir,sheet_name = 'Bradyinfo')
    trial_no = vsinfo.shape[0]
    
    roi = eval(bradyinfo['roi'][0])
    total_pixel_no = roi[2]*roi[3]

    for ii in range(0,len(visuomotor_list)):
        locals()['Index_'+str(visuomotor_list[ii])] = []
    for t in range(0,trial_no):
        if bradyinfo['Good_Pixel_Number'][t]/total_pixel_no>0.05:
            if vsinfo.Left_Stimulus_Type[t] == 'n' and vsinfo.Right_Stimulus_Type[t] == 'n':
                Index_ns.append(t+1)
            elif vsinfo.Left_Stimulus_Type[t] == 'm' or vsinfo.Right_Stimulus_Type[t] == 'm':
                if vsinfo.Left_Stimulus_Size[t] + vsinfo.Right_Stimulus_Size[t] == 15:
                    Index_dot15.append(t+1)
                if vsinfo['Left_Color'][t]=='UV' or vsinfo['Right_Color'][t]=='UV':
                    Index_dot4.append(t+1)
            elif vsinfo['Left_Stimulus_Type'][t]=='l' or vsinfo['Right_Stimulus_Type'][t]=='l':
                Index_looming.append(t+1)
    
    for VMV in visuomotor_list[1:4]:
        print(VMV)
        index_temp = locals()['Index_'+VMV]
        for ROI_i in idx_fish:
            np_trace_temp = np.zeros((len(index_temp),time_len-10))
            np_avg_trace_temp = np.zeros((len(index_temp),time_len-10))
            for t_i in range(0,len(index_temp)):
                np_trace_temp[t_i,:] = np_trace_no_esc[ROI_i,index_temp[t_i]*time_len-time_len+10:index_temp[t_i]*time_len]
                np_avg_trace_temp[t_i] = np_motor_avg_trace[ROI_i,index_temp[t_i]*time_len-time_len+10:index_temp[t_i]*time_len]
#             np_trace_temp = (np_trace_temp-np.min(np_trace_temp))/(np.max(np_trace_temp)-np.min(np_trace_temp))
#             np_avg_trace_temp = (np_avg_trace_temp-np.min(np_avg_trace_temp))/(np.max(np_avg_trace_temp)-np.min(np_avg_trace_temp))
            temp_sensory_idx = np.sqrt(np.var(np_avg_trace_temp)/np.var(np_trace_temp))
            df_corr.loc[ROI_i,'sensory_idx_'+VMV] = temp_sensory_idx

In [ ]:
responsiveness_threshold = 0.9
periodicity_threshold = 0.85
size = 2

for VMV in visuomotor_list[1:4]:
    locals()[VMV+'_sensory_idx'] = []

for main_dir,fish,fish_i in zip(main_dir_list,fish_index_list,range(0,len(fish_index_list))):
    date_no = main_dir.split('\\')[-1]
    print(date_no+fish)
    idx_fish = np.where(np.logical_and(df_corr['Date'] == int(date_no),df_corr['Fish'] == fish))[0]
    
    # read bradyinfo and vsinfo
    vsinfo = pd.read_excel(main_dir+'\\behavior\\'+fish+'//vsinfo_'+date_no+'_'+fish+'.xlsx')
    bradyinfo_dir = glob.glob(main_dir+'\\behavior\\'+fish+'\\Bradyinfo*'+fish+'.xlsx')[0]
                                  
    bradyinfo = pd.read_excel(bradyinfo_dir,sheet_name = 'Bradyinfo')
    trial_no = vsinfo.shape[0]
    
    roi = eval(bradyinfo['roi'][0])
    total_pixel_no = roi[2]*roi[3]

    for ii in range(0,len(visuomotor_list)):
        locals()['Index_'+str(visuomotor_list[ii])] = []
    for t in range(0,trial_no):
        if bradyinfo['Good_Pixel_Number'][t]/total_pixel_no>0.05:
            if vsinfo.Left_Stimulus_Type[t] == 'n' and vsinfo.Right_Stimulus_Type[t] == 'n':
                Index_ns.append(t+1)
            elif vsinfo.Left_Stimulus_Type[t] == 'm' or vsinfo.Right_Stimulus_Type[t] == 'm':
                if vsinfo.Left_Stimulus_Size[t] + vsinfo.Right_Stimulus_Size[t] == 15:
                    Index_dot15.append(t+1)
                if vsinfo['Left_Color'][t]=='UV' or vsinfo['Right_Color'][t]=='UV':
                    Index_dot4.append(t+1)
            elif vsinfo['Left_Stimulus_Type'][t]=='l' or vsinfo['Right_Stimulus_Type'][t]=='l':
                Index_looming.append(t+1)

    for VMV,VMV_i in zip(visuomotor_list[1:4],range(0,3)):
        idx_responsive_ROI = np.where(df_corr.loc[idx_fish,VMV].rank(pct=True)>responsiveness_threshold)[0]
        idx_responsive_ROI = idx_fish[idx_responsive_ROI]
        idx_periodic_ROI = np.where(df_corr.loc[idx_responsive_ROI,'sensory_idx_'+VMV].rank(pct=True)>periodicity_threshold)[0]
        idx_periodic_ROI = idx_responsive_ROI[idx_periodic_ROI]
#         high_motor_idx = []
#         for sensory_motor in sensory_motor_list:
#             high_motor_idx_temp = np.where(df_corr.loc[idx_fish,sensory_motor+'_motor_res_r'].rank(pct=True)>motor_threshold)[0]
#             high_motor_idx.extend(list(idx_fish[high_motor_idx_temp]))
#         idx_periodic_ROI = list(set(idx_periodic_ROI)-set(np.unique(high_motor_idx)))
        locals()[VMV+'_sensory_idx'].extend(list(idx_periodic_ROI))

    
# spatial p value filtering
random_no = 5000

for VMV,color in zip(visuomotor_list[1:4],['crimson','dodgerblue','lime']):
    print(VMV)
    locals()['filtered_sensory_idx_'+VMV] = []
    
    df_corr_VMV = df_corr.loc[locals()[VMV+'_sensory_idx'],:]
    
    df_id_list = []
    for main_dir,temp_fish in zip(main_dir_list,fish_index_list):
        temp_date = main_dir.split('\\')[-1]
        df_id = str(temp_date)+temp_fish
        df_id_list.append(df_id)
        locals()['df_'+df_id+'_'+VMV] = df_corr_VMV[np.logical_and(df_corr_VMV['Date'] == int(temp_date),df_corr_VMV['Fish'] == temp_fish)]
    
    for i in locals()[VMV+'_sensory_idx']:
        
        temp_coor = list(df_corr.loc[i,['reg_x','reg_y','reg_z']]) 
        temp_date = df_corr['Date'][i]
        temp_fish = df_corr['Fish'][i]
        temp_df_id = str(temp_date)+str(temp_fish)
        
        dr_list = []
        #calculate dr of this neuron
        for df_id in df_id_list:
            if df_id != temp_df_id and locals()['df_'+df_id+'_'+VMV].shape[0]>0:
                temp_fish_coor_array = np.array(locals()['df_'+df_id+'_'+VMV].loc[:,['reg_x','reg_y','reg_z']])
                distance,index = spatial.KDTree(temp_fish_coor_array).query(temp_coor)
                dr_list.append(distance)
            if df_id == temp_df_id and locals()['df_'+df_id+'_'+VMV].shape[0]>1:
                temp_fish_coor_array = np.array(locals()['df_'+df_id+'_'+VMV].loc[:,['reg_x','reg_y','reg_z']])
                distance,index = spatial.KDTree(temp_fish_coor_array).query(temp_coor,k=2)
                dr_list.append(distance[1])
        if len(dr_list)>3:
            mean_dr = np.mean(reject_outliers(np.array(dr_list)))
            #calculate and fit dn normal distribution of this neuron

            random_sampling = np.random.choice(df_corr.shape[0],random_no, replace=True)
            temp_fish_coor_array_random = np.array(df_corr.loc[random_sampling,['reg_x','reg_y','reg_z']])
            dn = spatial.distance.cdist(np.array([temp_coor]*random_no),temp_fish_coor_array_random)
            mu,s = stats.norm.fit(dn)

            # reject the null hypothesis of random sampling if a neuron has a spatial P value < 0.025.
            if NormalDist(mu, s).cdf(mean_dr)<0.025:
                locals()['filtered_sensory_idx_'+VMV].append(i)

# MSI calculation and motor neuron selection

In [ ]:
for sensory_motor in sensory_motor_list:
    locals()['motor_surplus_idx_'+sensory_motor] = np.array([])
threshold = 0.97

for main_dir,fish in zip(main_dir_list,fish_index_list):
    date_no = main_dir.split('\\')[-1]
    print(date_no+fish)
    idx_fish = np.where(np.logical_and(df_corr['Date'] == int(date_no),df_corr['Fish'] == fish))[0]
    
    # read bradyinfo and vsinfo
    vsinfo = pd.read_excel(main_dir+'\\behavior\\'+fish+'//vsinfo_'+date_no+'_'+fish+'.xlsx')
    bradyinfo_dir = glob.glob(main_dir+'\\behavior\\'+fish+'\\Bradyinfo*'+fish+'.xlsx')[0]
                                  
    bradyinfo = pd.read_excel(bradyinfo_dir,sheet_name = 'Bradyinfo')
    trial_no = vsinfo.shape[0]
    
    roi = eval(bradyinfo['roi'][0])
    total_pixel_no = roi[2]*roi[3]
    
    
    # reorganize freezing bouts
    bradyinfo['Freezing_bouts'] = '[]'
    for t in range(0,trial_no):
        temp_freezing_bouts = []
        temp_brady_bouts = eval(bradyinfo['Brady_bouts'][t])
        temp_movement_bouts = eval(bradyinfo['Escape_bouts'][t])+eval(bradyinfo['Spontaneous_bouts'][t])+eval(bradyinfo['Eye_Convergence_bouts'][t])
        for brady_bout in temp_brady_bouts:
            brady_bout  = [b_i * 2 for b_i in brady_bout]
            temp_overlap = 0
            for movement_bout in temp_movement_bouts:
                temp_overlap = temp_overlap+getOverlap(brady_bout,movement_bout)
            if temp_overlap==0:
                temp_freezing_bouts.append(brady_bout)
        bradyinfo['Freezing_bouts'][t] = temp_freezing_bouts
    
   # every VMV's trial index of this fish
    for ii in range(0,len(visuomotor_list)):
        if ii>3:
            locals()['timing_'+str(visuomotor_list[ii])] = []
        else:
            locals()['Index_'+str(visuomotor_list[ii])] = []
    Index_dot4_motor_PC = []
    Index_dot15_motor_Freezing = []
    Index_looming_motor_Freezing = []
    Index_dot15_motor_escape = []
    Index_looming_motor_escape = []
        
    for t in range(0,trial_no):
        timing_motor_Freezing_trial = []
        timing_motor_PC_trial = []
        timing_motor_escape_trial = []
        if bradyinfo['Good_Pixel_Number'][t]/total_pixel_no>0.05:
            if vsinfo.Left_Stimulus_Type[t] == 'n' and vsinfo.Right_Stimulus_Type[t] == 'n':
                Index_ns.append(t+1)
            else:
                if vsinfo.Left_Stimulus_Type[t] == 'm' or vsinfo.Right_Stimulus_Type[t] == 'm':
                    if vsinfo.Left_Stimulus_Size[t] + vsinfo.Right_Stimulus_Size[t] == 15:
                        Index_dot15.append(t+1)
                        # Index_dot15_motor_escape
                        if bradyinfo['Escape_bouts'][t]!='[]':
                            timing_motor_escape_trial = []
                            temp_ind = False
                            for escape_bout in eval(bradyinfo['Escape_bouts'][t]):
                                if escape_bout[0]>1800 and escape_bout[0]<2600:
                                    temp_ind = True
                                    timing_motor_escape_trial.append(round(escape_bout[0]/fps_behavior*fps-1))
                            if temp_ind:
                                Index_dot15_motor_escape.append(t+1)
                        # Index_dot15_motor_Freezing
                        if len(bradyinfo['Freezing_bouts'][t])!=0:
                            temp_ind = False
                            for freezing_bout in bradyinfo['Freezing_bouts'][t]:
                                if freezing_bout[0]>1800 and freezing_bout[0]<2600:
                                    temp_ind = True
                                    timing_motor_Freezing_trial.append(round(freezing_bout[0]/fps_behavior*fps-1))
                            if temp_ind:
                                Index_dot15_motor_Freezing.append(t+1)
                        
                    if vsinfo['Left_Color'][t]=='UV' or vsinfo['Right_Color'][t]=='UV':
                        Index_dot4.append(t+1)
                        # Index_dot4_motor_PC
                        if bradyinfo['Eye_Convergence_bouts'][t]!='[]':
                            temp_ind = False
                            for pc_bout in eval(bradyinfo['Eye_Convergence_bouts'][t]):
                                if pc_bout[0]>1800 and pc_bout[0]<2600:
                                    temp_ind = True
                                    timing_motor_PC_trial.append(round(pc_bout[0]/fps_behavior*fps-1))
                            if temp_ind:
                                Index_dot4_motor_PC.append(t+1)
                if vsinfo['Left_Stimulus_Type'][t]=='l' or vsinfo['Right_Stimulus_Type'][t]=='l':
                    Index_looming.append(t+1)
                    # Index_looming_motor_escape
                    if bradyinfo['Escape_bouts'][t]!='[]':
                        timing_motor_escape_trial = []
                        temp_ind = False
                        for escape_bout in eval(bradyinfo['Escape_bouts'][t]):
                            if escape_bout[0]>1800 and escape_bout[0]<2600:
                                temp_ind = True
                                timing_motor_escape_trial.append(round(escape_bout[0]/fps_behavior*fps-1))
                        if temp_ind:
                            Index_looming_motor_escape.append(t+1)
                    # Index_looming_motor_Freezing
                    if len(bradyinfo['Freezing_bouts'][t])!=0:
                        temp_ind = False
                        for freezing_bout in bradyinfo['Freezing_bouts'][t]:
                            if freezing_bout[0]>1800 and freezing_bout[0]<2600:
                                temp_ind = True
                                timing_motor_Freezing_trial.append(round(freezing_bout[0]/fps_behavior*fps-1))
                        if temp_ind:
                            Index_looming_motor_Freezing.append(t+1)
                    
        timing_motor_Freezing.append(timing_motor_Freezing_trial)
        timing_motor_PC.append(timing_motor_PC_trial)
        timing_motor_escape.append(timing_motor_escape_trial)
    
    for sensory_motor in sensory_motor_list:
        if sensory_motor == 'dot15_motor_Freezing':
            index_temp = Index_dot15
            regressor_freezing_concat = np.array([])
            temp_activity_concat_motor_surplus = np.array([[]]).reshape((idx_fish.shape[0],0))
            for t in index_temp:
                temp_ind = True
                if bradyinfo['Escape_bouts'][t-1]!='[]':
                    for escape_bout in eval(bradyinfo['Escape_bouts'][t-1]):
                        if escape_bout[0]>1800 and escape_bout[0]<2600:
                            temp_ind = False
                if temp_ind:   
                    temp_activity_motor_surplus = np_motor_surplus_trace[idx_fish,t*time_len-time_len:t*time_len]
                    if t in locals()['Index_'+sensory_motor]:
                        if len(bradyinfo['Freezing_bouts'][t-1])!=0:
                            listFreezing = np.zeros(time_len)
                            for motor_i_start in timing_motor_Freezing[t-1]:
                                listFreezing[motor_i_start:motor_i_start+2] = 1
                            regressor_freezing = gen_regressor(listFreezing,7)
                            regressor_freezing = (regressor_freezing-min(regressor_freezing))/(max(regressor_freezing)-min(regressor_freezing))
                        else:
                            raise SystemExit('There are something wrong with the motor info')
                    else:
                        regressor_freezing = np.zeros(time_len)
                    temp_activity_concat_motor_surplus = np.concatenate((temp_activity_concat_motor_surplus,temp_activity_motor_surplus),axis =1)
                    regressor_freezing_concat = np.concatenate((regressor_freezing_concat,regressor_freezing))


            corr_motor_surplus = np.zeros(temp_activity_concat_motor_surplus.shape[0])
            for temp_activity_i in range(0,temp_activity_concat_motor_surplus.shape[0]):
                res = pearsonr(temp_activity_concat_motor_surplus[temp_activity_i,:],regressor_freezing_concat)
                corr_motor_surplus[temp_activity_i] = res[0]
            df_corr.loc[idx_fish,sensory_motor+'_motor_surplus_r'] = corr_motor_surplus

        elif sensory_motor == 'looming_motor_escape':
            index_temp = Index_looming
            regressor_escape_concat = np.array([])
            temp_activity_concat_motor_surplus = np.array([[]]).reshape((idx_fish.shape[0],0))
            for t in index_temp:
                temp_ind = True
                if len(bradyinfo['Freezing_bouts'][t-1])!=0:
                    for freezing_bout in bradyinfo['Freezing_bouts'][t-1]:
                        if freezing_bout[0]>1800 and freezing_bout[0]<2600:
                            temp_ind = False
                if temp_ind:
                    temp_activity_motor_surplus = np_motor_surplus_trace[idx_fish,t*time_len-time_len:t*time_len]
                    if t in locals()['Index_'+sensory_motor]:
                        if bradyinfo['Escape_bouts'][t-1]!='[]':
                            listescape = np.zeros(time_len)
                            for motor_i_start in timing_motor_escape[t-1]:
                                listescape[motor_i_start:motor_i_start+2] = 1
                            regressor_escape = gen_regressor(listescape,7)
                            regressor_escape = (regressor_escape-min(regressor_escape))/(max(regressor_escape)-min(regressor_escape))
                        else:
                            raise SystemExit('There are something wrong with the motor info')
                    else:
                        regressor_escape = np.zeros(time_len)
                    temp_activity_concat_motor_surplus = np.concatenate((temp_activity_concat_motor_surplus,temp_activity_motor_surplus),axis =1)
                    regressor_escape_concat = np.concatenate((regressor_escape_concat,regressor_escape))


            corr_motor_surplus = np.zeros(temp_activity_concat_motor_surplus.shape[0])
            for temp_activity_i in range(0,temp_activity_concat_motor_surplus.shape[0]):
                res = pearsonr(temp_activity_concat_motor_surplus[temp_activity_i,:],regressor_escape_concat)
                corr_motor_surplus[temp_activity_i] = res[0]
            df_corr.loc[idx_fish,sensory_motor+'_motor_surplus_r'] = corr_motor_surplus

        elif sensory_motor == 'dot4_motor_PC':
            index_temp = Index_dot4    
            regressor_pc_concat = np.array([])
            temp_activity_concat_motor_surplus = np.array([[]]).reshape((idx_fish.shape[0],0))
            for t in index_temp:
                temp_activity_motor_surplus = np_motor_surplus_trace[idx_fish,t*time_len-time_len:t*time_len]
                if t in locals()['Index_'+sensory_motor]:
                    if bradyinfo['Eye_Convergence_bouts'][t-1]!='[]':
                        listpc = np.zeros(time_len)
                        for motor_i_start in timing_motor_PC[t-1]:
                            listpc[motor_i_start:motor_i_start+2] = 1
                        regressor_pc = gen_regressor(listpc,7)                        
                        regressor_pc = (regressor_pc-min(regressor_pc))/(max(regressor_pc)-min(regressor_pc))

                    else:
                        raise SystemExit('There are something wrong with the motor info')
                else:
                    regressor_pc = np.zeros(time_len)
                temp_activity_concat_motor_surplus = np.concatenate((temp_activity_concat_motor_surplus,temp_activity_motor_surplus),axis =1)
                regressor_pc_concat = np.concatenate((regressor_pc_concat,regressor_pc))


            corr_motor_surplus = np.zeros(temp_activity_concat_motor_surplus.shape[0])
            for temp_activity_i in range(0,temp_activity_concat_motor_surplus.shape[0]):
                res = pearsonr(temp_activity_concat_motor_surplus[temp_activity_i,:],regressor_pc_concat)
                corr_motor_surplus[temp_activity_i] = res[0]
            df_corr.loc[idx_fish,sensory_motor+'_motor_surplus_r'] = corr_motor_surplus
    
    for sensory_motor in sensory_motor_list:
        if sensory_motor!='looming_motor_escape' or fish!='F6_21_00':
            motor_surplus_idx = df_corr.loc[idx_fish,sensory_motor+'_motor_surplus_r'].rank(pct=True)>threshold
            locals()['motor_surplus_idx_'+sensory_motor] = np.concatenate((locals()['motor_surplus_idx_'+sensory_motor],idx_fish[motor_surplus_idx]))


In [ ]:
# sensorymotor neuron selection and spatial p value filtering
threshold_ = 0.97
random_no = 5000
size = 2

for sensory_motor in sensory_motor_list:
    locals()['sensorymotor_idx_'+sensory_motor] = []

    df_corr_sensory_motor = df_corr.loc[locals()['motor_surplus_idx_'+sensory_motor],:]

    df_id_list = []
    if sensory_motor == 'looming_motor_escape':
        for main_dir,temp_fish in zip(main_dir_list[:2]+main_dir_list[-4:],fish_index_list[:2]+fish_index_list[-4:]):
            temp_date = main_dir.split('\\')[-1]
            df_id = str(temp_date)+temp_fish
            df_id_list.append(df_id)
            locals()['df_'+df_id+'_'+sensory_motor] = df_corr_sensory_motor[np.logical_and(df_corr_sensory_motor['Date'] == int(temp_date),df_corr_sensory_motor['Fish'] == temp_fish)]

    else:
        for main_dir,temp_fish in zip(main_dir_list,fish_index_list):
            temp_date = main_dir.split('\\')[-1]
            df_id = str(temp_date)+temp_fish
            df_id_list.append(df_id)
            locals()['df_'+df_id+'_'+sensory_motor] = df_corr_sensory_motor[np.logical_and(df_corr_sensory_motor['Date'] == int(temp_date),df_corr_sensory_motor['Fish'] == temp_fish)]

    for i in locals()['motor_surplus_idx_'+sensory_motor]:

        temp_coor = list(df_corr.loc[i,['reg_x','reg_y','reg_z']]) 
        temp_date = df_corr['Date'][i]
        temp_fish = df_corr['Fish'][i]
        temp_df_id = str(temp_date)+str(temp_fish)

        dr_list = []
        #calculate dr of this neuron
        for df_id in df_id_list:
            if df_id != temp_df_id and locals()['df_'+df_id+'_'+sensory_motor].shape[0]>0:
                temp_fish_coor_array = np.array(locals()['df_'+df_id+'_'+sensory_motor].loc[:,['reg_x','reg_y','reg_z']])
                distance,index = spatial.KDTree(temp_fish_coor_array).query(temp_coor)
                dr_list.append(distance)
            if df_id == temp_df_id and locals()['df_'+df_id+'_'+sensory_motor].shape[0]>1:
                temp_fish_coor_array = np.array(locals()['df_'+df_id+'_'+sensory_motor].loc[:,['reg_x','reg_y','reg_z']])
                distance,index = spatial.KDTree(temp_fish_coor_array).query(temp_coor,k=2)
                dr_list.append(distance[1])
        if len(dr_list)>3:
            mean_dr = np.mean(reject_outliers(np.array(dr_list)))
            #calculate and fit dn normal distribution of this neuron

            random_sampling = np.random.choice(df_corr.shape[0],random_no, replace=True)
            temp_fish_coor_array_random = np.array(df_corr.loc[random_sampling,['reg_x','reg_y','reg_z']])
            dn = spatial.distance.cdist(np.array([temp_coor]*random_no),temp_fish_coor_array_random)
            mu,s = stats.norm.fit(dn)

            # reject the null hypothesis of random sampling if a neuron has a spatial P value < 0.025.
            if NormalDist(mu, s).cdf(mean_dr)<0.025:
                locals()['sensorymotor_idx_'+sensory_motor].append(int(i))
                
                
for main_dir,fish in zip(main_dir_list,fish_index_list):
    date_no = main_dir.split('\\')[-1]
    idx_fish = np.where(np.logical_and(df_corr['Date'] == int(date_no),df_corr['Fish'] == fish))[0]

    for sensory_motor in sensory_motor_list:
        df_corr.loc[idx_fish,'motor_idx_'+sensory_motor] = (df_corr.loc[idx_fish,sensory_motor+'_motor_surplus_r']-np.min(df_corr.loc[idx_fish,sensory_motor+'_motor_surplus_r']))/(np.max(df_corr.loc[idx_fish,sensory_motor+'_motor_surplus_r'])-np.min(df_corr.loc[idx_fish,sensory_motor+'_motor_surplus_r']))